# Demo SingleRun
This notebook shows a single run consisting of the following four phases:
* sail empty
* loading
* sail full
* unloading

In [1]:
import datetime, time
import simpy

import shapely.geometry

import numpy as np
import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

In [2]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)


## Definition of Site

In [3]:
from_site = Site(
    env=my_env,
    name="Winlocatie",
    ID="6dbbbdf4-4589-11e9-a501-b469212bff5d",
    geometry=location_from_site,
    capacity=100000,
    level=100000,
)


to_site = Site(
    env=my_env,
    name="Dumplocatie",
    ID="6dbbbdf5-4589-11e9-82b2-b469212bff5c",
    geometry=location_to_site,
    capacity=100000,
    level=0,
)

## Definition of Vessels

In [4]:
hopper = TransportProcessingResource(
    env=my_env,
    name="Hopper 01",
    ID="6dbbbdf6-4589-11e9-95a2-b469212bff5b",
    geometry=location_from_site,
    loading_rate=0.00001,
    unloading_rate=0.00001,
    capacity=4,
    compute_v=lambda x: 10,
)

## Defenition of the activities

In [5]:
load_actitivy = model.MoveActivity(
    env=my_env,
    name="sailing empty",
    registry=registry,
    mover=hopper,
    destination=from_site,
    duration=1,
)
sail_empt_actitivy = model.ShiftAmountActivity(
    env=my_env,
    name="Transfer MP",
    registry=registry,
    processor=hopper,
    origin=from_site,
    destination=hopper,
    amount=4,
    duration=1,
)
unload_actitivy = model.MoveActivity(
    env=my_env,
    name="sailing filler",
    registry=registry,
    mover=hopper,
    destination=to_site,
    duration=1,
)
sail_full_actitivy = model.ShiftAmountActivity(
    env=my_env,
    name="Transfer TP",
    registry=registry,
    processor=hopper,
    origin=hopper,
    destination=to_site,
    amount=4,
    duration=1,
)
wait__actitivy = model.BasicActivity(
    env=my_env,
    name="Basic activity",
    registry=registry,
    duration=1,
    additional_logs=[hopper],
)

subcycle_activity = model.SequentialActivity(
    env=my_env,
    name="Subcycle activity",
    registry=registry,
    sub_processes=[wait__actitivy],
)

single_run = [
    load_actitivy,
    sail_empt_actitivy,
    unload_actitivy,
    sail_full_actitivy,
    subcycle_activity
]


sequence_activity = model.SequentialActivity(
    env=my_env,
    name="Single run process",
    registry=registry,
    sub_processes=single_run,
)


while_activity = model.RepeatActivity(
    env=my_env,
    name="while",
    registry=registry,
    sub_processes=[sequence_activity],
    repetitions=20,
)

In [6]:
model.register_processes([while_activity])
my_env.run()

In [7]:
 plot.vessel_planning([while_activity, *single_run, sequence_activity, wait__actitivy])

In [8]:
 plot.vessel_planning([hopper, from_site, to_site])

In [9]:
mapping = {
    act.id:act.name
    for act in [while_activity, *single_run, sequence_activity, wait__actitivy]
}
df = plot.get_log_dataframe(while_activity, [while_activity, *single_run, sequence_activity, wait__actitivy])
df.ref = df.ref.replace(mapping)
df = df.sort_values(by=['Timestamp','ActivityState'])

cols = list(df.keys())
df['count'] = df.groupby(cols)[cols[0]].transform('size')
df = df.drop_duplicates().reset_index(drop=True).fillna(1.0)
print(all(df['count'] == 1))
df.tail(60)

False


,Activity,Timestamp,ActivityState,type,ref,count
210,while,1970-01-01 00:01:20,STOP,subprocess,sailing filler,15.0
211,while,1970-01-01 00:01:20,STOP,subprocess,Transfer TP,15.0
212,while,1970-01-01 00:01:20,STOP,subprocess,Subcycle activity,15.0
213,while,1970-01-01 00:01:20,STOP,subprocess,Basic activity,120.0
214,while,1970-01-01 00:01:25,START,subprocess,sailing empty,16.0
215,while,1970-01-01 00:01:25,START,subprocess,Transfer MP,16.0
216,while,1970-01-01 00:01:25,START,subprocess,sailing filler,16.0
217,while,1970-01-01 00:01:25,START,subprocess,Transfer TP,16.0
218,while,1970-01-01 00:01:25,START,subprocess,Subcycle activity,16.0
219,while,1970-01-01 00:01:25,START,subprocess,Basic activity,136.0


In [10]:
mapping = {
    act.id:act.name
    for act in [while_activity, *single_run, sequence_activity, wait__actitivy]
}
df = plot.get_log_dataframe(sequence_activity, [while_activity, *single_run, sequence_activity, wait__actitivy])
df.ref = df.ref.replace(mapping)
df = df.sort_values(by=['Timestamp','ActivityState'])

cols = list(df.keys())
df['count'] = df.groupby(cols)[cols[0]].transform('size')
df = df.drop_duplicates().reset_index(drop=True).fillna(1.0)
print(all(df['count'] == 1))
df.head(60)

True


,Activity,Timestamp,ActivityState,type,ref,count
0,Single run process,1970-01-01 00:00:00,START,1,1,1.0
1,Single run process,1970-01-01 00:00:00,START,subprocess,sailing empty,1.0
2,Single run process,1970-01-01 00:00:01,START,subprocess,Transfer MP,1.0
3,Single run process,1970-01-01 00:00:01,STOP,subprocess,sailing empty,1.0
4,Single run process,1970-01-01 00:00:02,START,subprocess,sailing filler,1.0
5,Single run process,1970-01-01 00:00:02,STOP,subprocess,Transfer MP,1.0
6,Single run process,1970-01-01 00:00:03,START,subprocess,Transfer TP,1.0
7,Single run process,1970-01-01 00:00:03,STOP,subprocess,sailing filler,1.0
8,Single run process,1970-01-01 00:00:04,START,subprocess,Subcycle activity,1.0
9,Single run process,1970-01-01 00:00:04,STOP,subprocess,Transfer TP,1.0
